In [1]:
!pip install yt-dlp moviepy flask pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.7 MB/s eta 0:00:00


In [2]:
%%writefile 102303764.py

import sys
import os
import yt_dlp
from moviepy.editor import AudioFileClip, concatenate_audioclips

def error_exit(msg):
    print("Error:", msg)
    sys.exit(1)

def main():

    if len(sys.argv) != 5:
        error_exit("Usage: python 102303764.py <SingerName> <NumberOfVideos> <AudioDuration> <OutputFileName>")

    singer = sys.argv[1]

    try:
        num = int(sys.argv[2])
        duration = int(sys.argv[3])
    except:
        error_exit("NumberOfVideos and AudioDuration must be integers.")

    output = sys.argv[4]

    if num <= 10:
        error_exit("NumberOfVideos must be > 10")

    if duration <= 20:
        error_exit("AudioDuration must be > 20")

    os.makedirs("downloads", exist_ok=True)

    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": "downloads/%(id)s.%(ext)s",
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
        }],
        "quiet": True
    }

    query = f"ytsearch{num}:{singer}"

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([query])

    clips = []

    for f in os.listdir("downloads"):
        if f.endswith(".mp3"):
            clip = AudioFileClip(os.path.join("downloads", f)).subclip(0, duration)
            clips.append(clip)

    if not clips:
        error_exit("No clips processed")

    final = concatenate_audioclips(clips)
    final.write_audiofile(output)

    print("Mashup created:", output)

if __name__ == "__main__":
    main()

Writing 102303764.py


In [3]:
!python 102303764.py "Sharry Maan" 12 30 output.mp3

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
A

In [4]:
from google.colab import files
files.download("output.mp3")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from flask import Flask, request, send_file
import os
import yt_dlp
from moviepy.editor import AudioFileClip, concatenate_audioclips

app = Flask(__name__)

@app.route("/", methods=["GET","POST"])
def index():

    if request.method == "POST":

        singer = request.form["singer"]
        num = int(request.form["num"])
        duration = int(request.form["duration"])

        os.makedirs("downloads", exist_ok=True)

        ydl_opts = {
            "format": "bestaudio/best",
            "outtmpl": "downloads/%(id)s.%(ext)s",
            "postprocessors": [{
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
            }],
            "quiet": True
        }

        query = f"ytsearch{num}:{singer}"

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([query])

        clips = []

        for f in os.listdir("downloads"):
            if f.endswith(".mp3"):
                clip = AudioFileClip(os.path.join("downloads", f)).subclip(0, duration)
                clips.append(clip)

        final = concatenate_audioclips(clips)
        output = "mashup.mp3"
        final.write_audiofile(output)

        return send_file(output, as_attachment=True)

    return '''
<form method="POST">
Singer:<input name="singer"><br>
Videos:<input name="num"><br>
Duration:<input name="duration"><br>
<button>Generate Mashup</button>
</form>
'''

if __name__ == "__main__":
    app.run()